## Testing out some code to hit the NHL API


Resources

* https://towardsdatascience.com/nhl-analytics-with-python-6390c5d3206d
* https://nhl-api-explorer.herokuapp.com/
* https://gitlab.com/dword4/nhlapi
* https://gitlab.com/dword4/nhlapi/-/blob/master/stats-api.md#game-ids
* https://github.com/dword4/nhlapi

Game IDs
* The first 4 digits identify the season of the game (ie. 2017 for the 2017-2018 season). 
* The next 2 digits give the type of game, where 01 = preseason, 02 = regular season, 03 = playoffs, 04 = all-star. 
* The final 4 digits identify the specific game number. 
* For regular season and preseason games, this ranges from 0001 to the number of games played. (1271 for seasons with 31 teams (2017 and onwards) and 1230 for seasons with 30 teams). 
* For playoff games, the 2nd digit of the specific number gives the round of the playoffs, the 3rd digit specifies the matchup, and the 4th digit specifies the game (out of 7).

In [131]:
# Load dependencies
import requests
import pickle
import pandas as pd

In [194]:
# Set up the API call variables
year = '2021'
season_type = '02' 
max_game_id = 20
boxscore_url = '/boxscore'
livefeed_url = '/feed/live'
game_url = 'http://statsapi.web.nhl.com/api/v1/game/'

In [205]:
# Test hitting the boxscore and live feed endpoints, save as JSON files
def request_get_data(url, 
                    year, 
                    season_type, 
                    max_game_id, 
                    endpoint,
                    empty_list = [],
                    zfill = 4, 
                    multiple_games = True):

    # If multiple_games param is true, the get data for n number of games (uses for loop)
    if multiple_games == True:    
        for i in range(0, max_game_id):
            r = requests.get(url = str(url) + str(year) + str(season_type) + str(max_game_id).zfill(zfill) + str(endpoint))
            data = r.json()
            empty_list.append(data)

    # If multiple_games param is not true, then get data for the 1 game (no for loop)
    else:
        r = requests.get(url = str(url) + str(year) + str(season_type) + str(max_game_id).zfill(zfill) + str(endpoint))
        data = r.json()
        empty_list.append(data)

    # Rename to game_data_json
    game_data_list = empty_list

    return game_data_list

# Function that takes in a list, converts to json, then saves a copy of that json file in the local folder
def save_as_json(list, 
                 file_name = 'data'):
    
    # Save as json file in same folder the notebook is run (relative)
    with open(f'{file_name}.json', 'w') as f:
        json.dump(list, f)
    
    return j_data

In [206]:
# For a single game, save json files for both the boxscore and livefeed api
single_game_id = 21

# Boxscore
boxscore_list = request_get_data(game_url, 
                                        year, 
                                        season_type, 
                                        single_game_id, 
                                        boxscore_url,
                                        multiple_games = False)
boxscore_json = save_as_json(boxscore_list, 'test_data/boxscore')

# Livefeed
livefeed_list = request_get_data(game_url, 
                                        year, 
                                        season_type, 
                                        single_game_id, 
                                        livefeed_url,
                                        multiple_games = False)
livefeed_json = save_as_json(livefeed_list, 'test_data/livefeed')

In [ ]:
# For every player in the the boxscore, create a summary of their performance by season and other time periods


In [190]:
r = requests.get(url='http://statsapi.web.nhl.com/api/v1/game/'
    + year + season_type + str(max_game_id).zfill(4)+'/feed/live')
data = r.json()
game_data.append(data)

import json
with open('data.json', 'w') as f:
    json.dump(data, f)

In [172]:
for i in range(0, max_game_ID):
    r = requests.get(url='http://statsapi.web.nhl.com/api/v1/game/'
        + year + season_type + str(i).zfill(4)+'/feed/live')
    data = r.json()
    game_data.append(data)

{'messageNumber': 11, 'message': 'Invalid Request with value: ID'}

In [177]:
# Loop over the counter and format the API call
for i in range(0, max_game_ID):
    r = requests.get(url='http://statsapi.web.nhl.com/api/v1/game/'
        + year + season_type + str(i).zfill(4)+'/boxscore')
    data = r.json()
    game_data.append(data)


In [175]:
r = requests.get(url = 'http://statsapi.web.nhl.com/api/v1/game/2021020001/boxscore')
r.json

<bound method Response.json of <Response [200]>>

## Analyzing player shooting data

In [134]:
import numpy as np 
import pandas as pd 
import pickle    
import matplotlib
import matplotlib.pyplot as plt
color_map = plt.cm.winter
from matplotlib.patches import RegularPolygon
import math
from PIL import Image
# Needed for custom colour mapping!
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import matplotlib.colors as mcolors

In [135]:
# Custom colour map
def color_converter(hex_code1, hex_code2):
    c = mcolors.ColorConverter()
    x = ListedColormap([c.to_rgb(hex_code1),c.to_rgb(hex_code2)])
    return x

# Define positive and negative
positive_cm = color_converter('#e1e5e5', '#d63b36')
negative_cm = color_converter('#e1e5e5', '#28aee4')

In [136]:
# Do some dictionary initialisation to hold our cleaned and condensed league data
league_data = {};
league_data['Shot'] = {};
league_data['Shot']['x'] = [];
league_data['Shot']['y'] = [];
league_data['Goal'] = {};
league_data['Goal']['x'] = [];
league_data['Goal']['y'] = [];

In [137]:
# Only keep the events in the data for the Shot and Goals
event_types = ['Shot','Goal']

In [138]:
# First loop over the elements of game_data. Each one of these is an NHL game 
# ... and contains all of the game event data.

for data in game_data:
    # It is possible that the game data is not assigned to the dataset
    #... so to handle this we look for the key ‘liveData’ which 
    #... contains all of the data we are looking for, otherwise we continue
    if 'liveData' not in data:
        continue
    # Drilling down into the dataset to extract the play by play information for the game
    plays = data['liveData']['plays']['allPlays'] 

    for play in plays: # For each play
         for event in event_types:  # For each event (Shot,Goal)
            # If the play contains one of the events
            if play['result']['event'] in [event]:  
                # If the event contains coordinates
                if 'x' in play['coordinates']:
                    # Save the coordinates to the growing list
                    league_data[event]['x'].append(play['coordinates']['x'])
                    league_data[event]['y'].append(play['coordinates']['y'])

In [165]:
# Initialise the player dictionary
full_name = 'Kirill Kaprizov'
player_data = {};
player_data['Shot'] = {};
player_data['Shot']['x'] = [];
player_data['Shot']['y'] = [];
player_data['Goal'] = {};
player_data['Goal']['x'] = [];
player_data['Goal']['y'] = [];

In [140]:
# Same code as before
for play in plays:
    if 'players' in play:
        for player in play['players']:
            if player['player']['fullName'] in [full_name] and player['playerType'] in ["Shooter","Scorer"]:
                for event in event_types:  # For each event (Shot,Goal)
                    # If the play contains one of the events
                    if play['result']['event'] in [event]:  
                        # If the event contains coordinates
                        if 'x' in play['coordinates']:
                            # Save the coordinates to the growing list
                            league_data[event]['x'].append(play['coordinates']['x'])
                            league_data[event]['y'].append(play['coordinates']['y'])

## Calculating Basic Stats

In [142]:
# Get the total number of shots made by the player
player_total_shots = len(player_data['Shot']['x']) + len(player_data['Goal']['x'])
# Find the players goal score percentage
player_goal_pct = len(player_data['Goal']['x'])/player_total_shots
# Find the total number of shots taken in the league
league_total_shots = len(league_data['Shot']['x']) + len(league_data['Goal']['x'])
# Get the league percentage
if league_total_shots > 0:
    league_goal_pct = len(league_data['Goal']['x']) / league_total_shots
else
    league_goal_pct = .01
                
# Calculate the spread of the SOG (Shots on Goal) %
PL_e_spread = player_goal_pct-league_goal_pct

SyntaxError: invalid syntax (<ipython-input-142-e10c8eeb60d6>, line 10)

In [146]:
player_data

{'Shot': {'x': [], 'y': []}, 'Goal': {'x': [], 'y': []}}